In [1]:
import time
import sys
import os
import json
from openai import OpenAI
import fitz  # PyMuPDF
import textwrap
# load environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Get the API key
openai_api_key = os.getenv('OPENAI_API_KEY')

# Initialize the OpenAI client
client = OpenAI(api_key=openai_api_key)

# Define the assistant ID
assistant_id = "asst_CvyMd2gJ0Ymvg4y5YMnamfrp"

# Function to read PDF file content
def read_pdf(file_path):
    doc = fitz.open(file_path)
    full_text = []
    for page in doc:
        full_text.append(page.get_text())
    return '\n'.join(full_text)

# Load the candidate's resume from a PDF file
resume_content = read_pdf("JJ_Zhang_resume.pdf")

# Create a chat thread with the initial prompt
chat = client.beta.threads.create(
    messages=[
        {
            'role': 'user',
            'content': (
                'You are a hiring manager and need to hire a new senior data scientist. '
                'Please interview the candidate for technical skills and experience. '
                'Ask questions one by one, with follow-ups to dive deeper into the technical details '
                'of the machine learning projects he/she has worked on.'
            )
        }
    ]
)

# Function to interact with the assistant
def interact_with_assistant(chat_id):
    initial_message = True
    while True:
        # Retrieve and print the messages from the conversation
        message_response = client.beta.threads.messages.list(thread_id=chat_id)
        messages = message_response.data

        for message in messages:
            role = message.role
            content = message.content
            if isinstance(content, list):
                content = ' '.join([block.text.value for block in content])
            wrapped_content = textwrap.fill(content, width=80)
            if role != 'system' or not initial_message:
                print(f"{role.capitalize()}: {wrapped_content}\n")
            initial_message = False

        user_input = input("> ")  # Get user input
        if user_input.lower() == "exit":  # Check if the user wants to exit
            print("Exiting the conversation.")
            break  # Exit the loop

        # Send the user input as a new message in the thread
        client.beta.threads.messages.create(
            thread_id=chat_id,
            role='user',
            content=user_input
        )

        # Poll the run status until it is completed
        run = client.beta.threads.runs.create(thread_id=chat_id, assistant_id=assistant_id)
        while run.status != 'completed':
            run = client.beta.threads.runs.retrieve(thread_id=chat_id, run_id=run.id)
            print(f"Run status: {run.status}")
            time.sleep(1)

        print("Run completed")

# Start the interaction
interact_with_assistant(chat.id)

User: You are a hiring manager and need to hire a new senior data scientist. Please
interview the candidate for technical skills and experience. Ask questions one
by one, with follow-ups to dive deeper into the technical details of the machine
learning projects he/she has worked on.

Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: completed
Run completed
Assistant: Hello! Thank you for joining the interview today. I'm looking forward to
discussing your experience as a Data Scientist. To start, could you please give
me an overview of your most recent project related to machine learning? What was
the goal, and what were your specific contributions?

User: hello

User: You are a hiring manager and need to hire a new senior data scientist. Please
interview the candidate for technical skills and experience. Ask questions one
by one, with follow-ups to dive deep